# Modelling scheduling problem as constraint programming 

In this AIBT you learnt how to model a combinatorial optimisation problem using either Constraint Programming or Mixed Integer Linear programming paradigm. https://www.xoolive.org/optim4ai/
In this notebook you will be able to apply your CP modelling abilities to model RCPSP problem. Please refer to first notebook and slide deck [prez_Complex_Scheduling.pdf] to understand what constraints should be implemented

We will minizinc language directly in the notebook. In order to do that, just need to call this pip command.

In [ ]:
!pip install iminizinc

In [ ]:
import sys, os
this_folder = os.getcwd()
sys.path.append(os.path.join(this_folder, "discrete_optimisation/"))
import skdecide.hub
%load_ext iminizinc

In [ ]:
from discrete_optimization.rcpsp.rcpsp_model import RCPSPModel, RCPSPSolution
from discrete_optimization.rcpsp.rcpsp_parser import files_available, parse_file

In [ ]:
file = [f for f in files_available if "j301_10.sm" in f][0]
model = parse_file(file)
print(model)

In [ ]:
# Data definition for minizinc...
index_in_minizinc = {model.tasks_list[i]: i+1 for i in range(model.n_jobs)}
max_time = model.horizon
n_res = len(model.resources_list)
rc = [model.get_max_resource_capacity(r) for r in model.resources_list]
n_tasks = model.n_jobs
d = [model.mode_details[t][1]["duration"] for t in model.tasks_list]
rr = [[model.mode_details[t][1][r] for t in model.tasks_list] for r in model.resources_list]
adj = [[False for t in model.tasks_list] for t in model.tasks_list]
for t in model.tasks_list:
    for s in model.successors[t]:
        adj[index_in_minizinc[t]-1][index_in_minizinc[s]-1] = True


In [ ]:
%%minizinc -m bind --v
int: max_time;
set of int: TIMES=0..max_time;
int: n_res;                     % The number of resources
set of int: Res = 1..n_res;     % The set of all resources
array [Res] of int: rc;         % The resource capabilities

% Tasks
%
int: n_tasks;                           % The number of tasks
set of int: Tasks = 1..n_tasks;         % The set of all tasks
array [Tasks] of int: d;                % The task durations
array [Res, Tasks] of int: rr ;         % The resource requirements
array [Tasks, Tasks] of bool: adj;      % Adjacency matrix
array [Tasks] of set of Tasks: suc = [{p|p in Tasks where adj[j,p]}|j in Tasks];

% Variable 
array [Tasks] of var TIMES: s;  % The start times
var TIMES: makespan      ;  % The project duration (makespan)

% WRITE YOUR CONSTRAINTS :) !!!
output ["\(successors)"];
solve satisfy;

In [ ]:
print(s)

## Correction (to hide)

In [ ]:
#%%minizinc -m bind --verbose --solver chuffed
#%load correction/nb2_rcpsp_mzn.mzn

In [ ]:
solution = RCPSPSolution(problem=model, 
                         rcpsp_schedule={model.tasks_list[i]: {"start_time": start[i],
                                                               "end_time": start[i]+d[i]}
                                         for i in range(model.n_jobs)})

In [ ]:
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view, plot_task_gantt
plot_ressource_view(model, solution)
plot_task_gantt(model, solution)